In [1]:
# 2022.07.04 iPython notebook script to extract TAPE sequences from 10X sequencing reads
# Junhong Choi


In [ ]:
# Importing in tools

import gzip
import subprocess
import os,sys,csv,re
import itertools
from collections import Counter
from collections import OrderedDict
from operator import itemgetter
from datetime import datetime
from optparse import OptionParser,OptionGroup

# Usefule modules
import numpy as np
import edlib


# Plot tools 
import pandas as pd

# Additional function
def reverse_complement(seq):
    return complement(seq[::-1])

def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)

In [12]:
# Building one-off white-list dictionary for the TargetBC:
bases = ['A','C','T','G']

TargetBCs_correct = ['']
with open('TargetBC_WL19.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        TargetBCs_correct.append(row[0])


TargetBC_WL = {}
for BC in TargetBCs_correct:
    for ii in range(0,len(BC)):
        for base in bases:
            if ii == 0:
                mismatch = base + BC[1:]
            elif ii < len(BC):
                mismatch = BC[0:ii] + base + BC[ii+1:]
            elif ii == len(BC):
                mismatch = BC[0:ii] + base
            try:
                TargetBC_WL[mismatch] = BC
            except KeyError:
                continue

                
TAPE1_sequence = 'TGATGGTGAGCACG'
TAPE1_WL = []
for ii in range(0,len(TAPE1_sequence)):
    for base in bases:
        if ii == 0:
            mismatch = base + TAPE1_sequence[1:]
        elif ii < len(TAPE1_sequence):
            mismatch = TAPE1_sequence[0:ii] + base + TAPE1_sequence[ii+1:]
        elif ii == len(TAPE1_sequence):
            mismatch = TAPE1_sequence[0:ii] + base
        
        TAPE1_WL.append(mismatch)

        

InsertBC_WL = ['']
with open('InsertBC_WL19.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        InsertBC_WL.append(row[0])

In [13]:
c = 0
InsertBC_filtered = 0
cell_TAPE = {}
InsertBC_table = {}
TargetBC_table = {}

# Opening the BAM extract file
# Has [cBC][mBC(TargetBC-5xTAPE)][n_reads][n_UMI][n_reads_filtered][n_UMI_filtered]
with open('CellByTape_10X_runAll_bamExtract_t2.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        if c == 0:
            c+=1
            continue
        if c > 0:
            if int(row[3]) > 1: # confirming threshold for UMI
                TargetBC = row[1][35:43]
                try:
                    TargetBC_table[TargetBC] += 1
                except KeyError:
                    TargetBC_table[TargetBC] = 1
                try:
                    TargetBC = TargetBC_WL[TargetBC]
                    TAPE = row[1][60:]
                    for TAPE_var in TAPE1_WL:
                        TAPE = TAPE.replace(TAPE_var, ',') # Usual TAPE-1
                    TAPE = TAPE.replace('TGATGGTGAGCGAAGGAAA','') # Frequent ending
                    TAPE = TAPE.replace('GGATGATG','GGA,TGATG') # Edit from chimeric ending 
                    TAPE = TAPE[:TAPE.rfind(',')] # Frequent ending
                    #TAPE = TAPE[:TAPE.find('TGATGGTGAGCCAA')] # Frequent ending
                    
                    #InsertBC WhiteList
                    InsertBC_WL_flag = 0
                    insert_idx = [_.start() for _ in re.finditer(',', TAPE)] 
                    prev_idx = 0
                    for idx in insert_idx:
                        if TAPE[prev_idx:idx] not in InsertBC_WL:
                            InsertBC_WL_flag += 1
                            InsertBC_filtered += 1
                        try:
                            InsertBC_table[TAPE[prev_idx:idx]] += 1
                        except KeyError:
                            InsertBC_table[TAPE[prev_idx:idx]] = 1
                        prev_idx = idx+1
                    if TAPE[prev_idx:] not in InsertBC_WL: #Taking care of the last InsertBC
                        InsertBC_WL_flag = 1
                        InsertBC_filtered += 1
                    try:
                        InsertBC_table[TAPE[prev_idx:]] += 1
                    except KeyError:
                        InsertBC_table[TAPE[prev_idx:]] = 1
            
                    # If all InsertBC matched the WhiteList,
                    if InsertBC_WL_flag == 0:
                        nEntries = TAPE.count(',')
                        TAPE = TAPE + ','*(4-nEntries)
                        cell_target = row[0]+','+TargetBC
                        TAPE_read = [TAPE, row[3]]
                        #print(row[0]+','+TargetBC+'-'+TAPE+','+str(row[3]))
                        c += 1
                        try:
                            past_read = cell_TAPE[cell_target][1]
                            if int(past_read) <= int(row[3]):
                                cell_TAPE[cell_target] = TAPE_read
                                

                        except KeyError:
                            cell_TAPE[cell_target] = TAPE_read
                except KeyError:
                    continue

print(InsertBC_filtered)

50729


In [14]:
with open('CellByTargetBC5xTAPE_runAll_processed_MOI19.csv', 'w', newline='') as f0:
    f0.write('Cell,TargetBC,Site1,Site2,Site3,Site4,Site5,nUMI\n')
    for key,value in cell_TAPE.items():
        f0.write(key+','+value[0]+','+str(value[1])+'\n')
        

In [9]:
with open('LTC1_MOI19_TargetBC_table.csv', 'w', newline='') as f0:
    f0.write('TargetBC,count\n')
    for key,value in TargetBC_table.items():
        f0.write(key+','+str(value)+'\n')